In [1]:
import pandas as pd
import math
import snappy
import sage

In [2]:
pd.set_option('display.max_colwidth', None)

### Table of Knot Invariants for $T(4,n)$
Knot signature, Alexander polynomial, Arf, upsilon, etc.

In [3]:
def symmetric_alexander_poly(K):
    alex = K.alexander_polynomial(norm=False)
    alex_sym = alex.shift(-min(alex.exponents()) // 2) 
    return alex_sym

In [4]:
T45 = snappy.Link('T(4,5)')
T45.alexander_polynomial()

t^12 - t^11 + t^8 - t^6 + t^4 - t + 1

In [5]:
symmetric_alexander_poly(T45)

t^-6 - t^-5 + t^-2 - 1 + t^2 - t^5 + t^6

In [6]:
# compute d_invariant for T(p,q) torus knots
def d_invariant(T_pq):
    alex_sym = symmetric_alexander_poly(T_pq)
    coeff_dict = alex_sym.dict()
    pos_exponents = [k for k in coeff_dict.keys() if k>0]
    
    d_T = 0
    for j in pos_exponents:
        d_T += j * coeff_dict[j]
    return d_T

### TODO - this seems wrong according to Fairchild, et al. Prop 3.9.
should be $d(5,6)=6$ since $q=6=5\cdot 1 + 1$ so $d(5,6)=6\cdot 1$

In [7]:
d_invariant(snappy.Link('T(5,6)'))

-3

In [8]:
def make_table(p, q_range=None):
    if q_range is None:
        q_range = (p+1, p+12)
        
    invariants = []
    for q in range(q_range[0], q_range[1]):
        if math.gcd(p,q) != 1: continue
    
        knot_name = f'T({p},{q})'
        knot = snappy.Link(knot_name)
        sage_knot = knot.sage_link()
    
        floer = knot.knot_floer_homology()
        #print(floer)
        
        invariants.append([q, knot_name, knot.signature(), sage_knot.arf_invariant(), floer['nu'], d_invariant(knot)])
        
    return pd.DataFrame(data=invariants, columns=['q', 'Knot', 'Signature', 'Arf', '$\\nu$', 'd'])

In [9]:
make_table(4)

,q,Knot,Signature,Arf,$\nu$,d
0,5,"T(4,5)",-8,1,6,3
1,7,"T(4,7)",-14,0,9,4
2,9,"T(4,9)",-16,0,12,4
3,11,"T(4,11)",-22,1,15,5
4,13,"T(4,13)",-24,1,18,7
5,15,"T(4,15)",-30,0,21,8


In [10]:
make_table(5)

,q,Knot,Signature,Arf,$\nu$,d
0,6,"T(5,6)",-16,1,10,-3
1,7,"T(5,7)",-16,0,12,-4
2,8,"T(5,8)",-20,1,14,-5
3,9,"T(5,9)",-24,0,16,-6
4,11,"T(5,11)",-24,0,20,-6
5,12,"T(5,12)",-28,1,22,-7
6,13,"T(5,13)",-32,0,24,-8
7,14,"T(5,14)",-32,1,26,-9
8,16,"T(5,16)",-40,1,30,-9


Observe that Remark 2.2 from Fairchild, et al. holds:

For odd $q$,
$$
\text{Arf}(T(p,q)) =
\begin{cases}
0 & p \text{ odd or } q\equiv\pm1\bmod 8, \\
1 & p \text { even and } q \equiv\pm 3\bmod 8.
\end{cases}
$$

## Lower bounds on $\gamma_4(T(4,n))$

Let's see for which $n$ does $\gamma_4(T(4,n))\ge 2$ using Proposition 2.3 from Fairchild, et al.

Proposition 2.3. Given a knot $K$ in $S^3$, if $\sigma(K)+4\text{Arf}(K)\equiv 4\bmod 8$ then $\gamma_4(K)\ge2$.

In [11]:
combined = df['Signature'] + 4 * df['Arf']
combined_mod = combined % 8
pd.DataFrame(data=zip(df['n'], combined, combined_mod),
             columns=['n', '$\\sigma(K)+4\\text{Arf}(K)$', '$\\sigma(K)+4\\text{Arf}(K)\\bmod8$'])

NameError: name 'df' is not defined

Now let's compute the lower bound in Theorem 2.6, again from Fairchild, et al.

For a knot $K$ in $S^3$,
$$
\left| \nu(K)+\frac{\sigma(K)}{2} \right| \le \gamma_4(K).
$$

In [ ]:
lb = abs(df['floer nu'] + 0.5 * df['Signature'])

In [ ]:
pd.DataFrame(data=zip(df['n'], lb), columns=['n', '$\\left| \\nu(K)+\\frac{\\sigma(K)}{2} \\right|$']).to_latex()

In [ ]:
from sympy import latex
col_names = ['n', '\\left| \\nu(K)+\\frac{\\sigma(K)}{2} \\right|']
col_names = [n
latex_col_names = map(lambda tc: '$'+latex(tc)+'$', col_names)


In [ ]:
list(latex_col_names)